### **ECON 323 FINAL PROJECT**

### **PARLIAMENTARY PARTICIPATION AND ELECTORAL SUCCESS**

##### INTRODUCTION

In this project, we aim to create a visualization notebook based on a ECON 490 paper. The paper explores the impact of parliamentary participation on electoral success particularly in the Canadian House of Commons. 

In this case, parliamentary participation is measured by participation in question period and statements made in the House of Commons and the private member bills introduced by the Member of Parliament. Electoral success is measured by whether or not the Member of Parliament was reelected in the next federal elections.

For this project, we will create a visualization notebook that focuses on:
- visually summarizing the variables 
- exploring relationships between the independent and dependent variables 

##### LOADING PACKAGES

We start by loading all the necessary packages:

In [1]:
import pandas as pd
import numpy as np

##### INTRODUCING THE DATA 

##### *Reading the Data*

We will first load in the data from csv files and rename columns so that the two dataframes can be concatenated.

The data in the csv files has been compiled from several sources:
- Elections Canada for reelection and riding information
- OurCommons website for information on Private Member's Bills, statements made and Cabinet ministers
- SQL Database from the openparliament.com website for gender, party and politician name
- Conservative.ca website for information on Shadow Cabinet
- Age and years in House of Commons from Parl.ca website 

The timeframe for this data is from the 2015 federal elections to the 2021 federal elections i.e. the information considers all parliamentary activity from the 42nd and 43rd Parliament.

In [26]:
# reading in csv data 
results15 = pd.read_csv("DATA/2015Results.csv")
results19 = pd.read_csv("DATA/2019Results.csv")

,Name,Party,Parliament,Politician ID,RidingID,Province,Gender,ranfor43,reelected43,QuestionPeriod,PvtMemBills,Statements,Backbencher,DOB1,AgeOnElectionDay,YearsServedAtTimeOfElection
0,Dean Allison,Conservative Party of Canada,42,5,35068,ON,M,Yes,Yes,42,0,19,0,1965-02-18,54.0,15.323288
1,Rona Ambrose,Conservative Party of Canada,42,6,48033,AB,F,No,.,457,1,3,0,1969-03-15,50.0,15.323288
2,David Anderson,Conservative Party of Canada,42,9,47002,SK,M,No,.,53,1,20,0,1957-08-15,62.0,18.909589
3,Charlie Angus,New Democratic Party,42,11,35107,ON,M,Yes,Yes,168,0,15,1,1962-11-14,56.0,15.323288
4,Larry Bagnell,Liberal Party of Canada,42,15,60001,YT,M,Yes,Yes,11,1,15,1,1949-12-19,69.0,16.350685


In [4]:
# renaming columns so that dataframes can be concatenated correctly 
Results15 = results15.rename(columns = {"RidingID":"Riding ID", "ranfor43":"RanForNE", "reelected43":"Reelected", "DOB1":"DOB", "YearsServedAtTimeOfElection":"YearsInHOC"})
Results19 = results19.rename(columns = {"Election":"Parliament", "ranfor44":"RanForNE", "reelected44":"Reelected", "StatementsByMembers":"Statements", "YearsServedAtElection":"YearsInHOC"})

In [27]:
# concatenating dataframes
Results_Full = pd.concat([Results15, Results19], ignore_index=True)
Results_Full.head()
# combining 2015 and 2019 datasets leds to MPs showing up twice -- doesn't matter for analysis because each row is just one observation - person itself doesnt matter

,Name,Party,Parliament,Politician ID,Riding ID,Province,Gender,RanForNE,Reelected,QuestionPeriod,PvtMemBills,Statements,Backbencher,DOB,AgeOnElectionDay,YearsInHOC
0,Dean Allison,Conservative Party of Canada,42,5,35068,ON,M,Yes,Yes,42,0,19,0,1965-02-18,54.0,15.323288
1,Rona Ambrose,Conservative Party of Canada,42,6,48033,AB,F,No,.,457,1,3,0,1969-03-15,50.0,15.323288
2,David Anderson,Conservative Party of Canada,42,9,47002,SK,M,No,.,53,1,20,0,1957-08-15,62.0,18.909589
3,Charlie Angus,New Democratic Party,42,11,35107,ON,M,Yes,Yes,168,0,15,1,1962-11-14,56.0,15.323288
4,Larry Bagnell,Liberal Party of Canada,42,15,60001,YT,M,Yes,Yes,11,1,15,1,1949-12-19,69.0,16.350685


##### *Creating Dummy Variables*

Next, we create dummy variables for the following categorical variables:
- Gender
- RanForNE (i.e. whether or not the MP ran for the next federal election)
- ReelectionNE (i.e. whether or not the MP was reelected in the next federal election)
- Party (i.e. whether or not a MP belong to the Conservative Party, Liberal Party or the New Democratic Party)

In [28]:
# creating a dataframe with columns that need to be converted to dummy variables
Results_Dummy = pd.DataFrame(data=Results_Full, columns=['Gender', 'RanForNE', 'Reelected', 'Party'])

# creating dummy variables for each column of the above dataset
Dummies = pd.get_dummies(Results_Dummy)
Dummies.head()

,Gender_F,Gender_M,RanForNE_No,RanForNE_Yes,Reelected_.,Reelected_No,Reelected_Yes,Party_Bloc Québécois,Party_Conservative Party of Canada,Party_Green Party of Canada,Party_Independent,Party_Liberal Party of Canada,Party_New Democratic Party
0,0,1,0,1,0,0,1,0,1,0,0,0,0
1,1,0,1,0,1,0,0,0,1,0,0,0,0
2,0,1,1,0,1,0,0,0,1,0,0,0,0
3,0,1,0,1,0,0,1,0,0,0,0,0,1
4,0,1,0,1,0,0,1,0,0,0,0,1,0


Then, we add the previously created dummy variables to the original dataframe. Just to clean up, we rename some columns for clarity and drop unnecessary columns.

In [ ]:
# adding newly created dummy variables to the original dataframe
Results = pd.concat([Results_Full, Dummies], axis = 1)

# removing unnecessary columns
Results = Results.drop(columns=['Gender', 'RanForNE', 'Reelected', 'Party', 'DOB', 'Gender_M', 'RanForNE_No', 'Reelected_.', 'Reelected_No', 'Party_Bloc Québécois', 'Party_Green Party of Canada', 'Party_Independent'])

# renaming columns for better understanding
Results = Results.rename(columns = {"Gender_F":"Gender", "RanForNE_Yes":"RanForNE", "Reelected_Yes":"Reelected", "Party_Conservative Party of Canada":"ConservativeMem", "Party_Liberal Party of Canada":"LiberalMem", "Party_New Democratic Party":"NDPMem"})

##### *Manipulating the Data*

- create a histogram here? to explain questionlog 

In [23]:
# creating a log of the QuestionPeriod variable
Results['QuestionsLog'] = np.log10(Results['QuestionPeriod'])

/Users/anahat/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Now just summarize the data - where we are at so far basically

##### VISUALIZATION: VARIABLES

##### *Reelection*

##### VISUALIZATION: RELATIONSHIPS

##### *Questions and Gender*